# Ernesto Antonio Reyes Ramírez

# Optimización

# Tarea 8

In [86]:
#Librerias
import numpy as np
import math
import copy
import time

### 1. Método de región de confianza basado en Dogleg

In [336]:
def Dogleg(x, fx, grad_fx, H_fx, Delta):
    pB = -np.linalg.solve(H_fx,grad_fx)
    if np.linalg.norm(pB) <= Delta:
        return pB
    
    pU = -((grad_fx.T@grad_fx)/(grad_fx.T@H_fx@grad_fx))*grad_fx
    
    if np.linalg.norm(pU) >= Delta:
        return (Delta / np.linalg.norm(grad_fx)) * grad_fx
    a = np.dot(pB - pU, pB - pU)
    b = 2 * np.dot(pU, pB - pU)
    c = np.dot(pU, pU) - Delta ** 2
    t = (-b + np.sqrt(b ** 2 - 4 * a * c)) / (2 * a)
    return pU + t * (pB - pU)
    
    
    
def Trust_region_Dogleg(f,grad_f,H_f,x0,Delta_0,Delta_max,eta,tol,ite):
    x = x0
    Delta = Delta_0
    for i in range(ite):
        g = grad_f(x)
        B = H_f(x)
        eps = 1e-6
        B += np.eye(B.shape[0]) * eps
        p = Dogleg(x, f(x), g, B, Delta)
        rho = (f(x) - f(x + p)) / (-np.dot(g, p) - 0.5 * np.dot(p, np.dot(B, p)))
        if rho < eta:
            Delta *= 0.25
        else:
            if rho > 0.75 and np.linalg.norm(p) == Delta:
                Delta = min(2 * Delta, Delta_max)
            if rho > eta:
                x += p
        if np.linalg.norm(g) < tol:
            break
            
    return x,i

### 2. Método de región de confianza: Newton-Cauchy alterno

In [340]:
def Trust_region_Newton_Cauchy(f,grad_f,H_f,x0,Delta_0,Delta_max,eta,tol,ite):
    x = copy.deepcopy(x0)
    n = x.shape[0]
    Delta = Delta_0
    
    for i in range(ite):
        # Cálculo del paso de Newton
        Bk = np.eye(n)
        gk = grad_f(x)
        Hk = H_f(x)
        eps = 1e-6
        Hk += np.eye(Hk.shape[0]) * eps
        dk = -np.linalg.solve(Hk, gk)
        if np.linalg.norm(dk) <= Delta:
            pk = dk
            rho_k = (f(x) - f(x + pk)) / (-f(x) - gk.T @ pk - 0.5 * pk.T @ Hk @ pk)
            if rho_k < 0.25:
                Delta = 0.25 * Delta
            else:
                if rho_k > 0.75 and np.linalg.norm(pk) == Delta:
                    Delta = min(2 * Delta, Delta_max)
        else:
            # Cálculo del paso de Cauchy
            pk = -Delta * gk / np.linalg.norm(gk)
            rho_k = (f(x) - f(x + pk)) / (-gk.T @ pk - 0.5 * pk.T @ Bk @ pk)
            if rho_k < 0.25:
                Delta = 0.25 * Delta
            else:
                if rho_k > 0.75:
                    Delta = min(2 * Delta, Delta_max)
        # Actualización de x
        if rho_k > eta:
            x = x + pk
            
        if np.linalg.norm(gk) < tol:
            break
    
    return x,i

### 3. Algoritmo RFTR 

In [326]:
def RFTR(func,grad_func,hess_func, x0, maxiter=100, eta1=0.05, eta2=0.9, gamma1=0.5, gamma2=2.0, eps=1e-6):
    x = x0.copy()
    n = x.shape[0]
    Delta = np.linalg.norm(x) / 10.0
    
    fval = func(x)
    grad = grad_func(x)
    Hess = hess_func(x)
    I = np.identity(n)
    Hk = Hess
    Sk = I
    
    for k in range(maxiter):
        # Calculas la region de confianza
        pk = -np.linalg.solve(Hk, grad)
        norm_pk = np.linalg.norm(pk)
        if norm_pk <= Delta:
            x_new = x + pk
        else:
            x_new = x + (Delta / norm_pk) * pk
        
        fval_new = func(x_new)
        rho_k = (fval - fval_new) / (-np.dot(grad, pk) - 0.5 * np.dot(pk, np.dot(Hess, pk)))
        
        if rho_k < eta1:
            Delta *= gamma1
        elif rho_k > eta2:
            Delta *= gamma2
        
        if rho_k > eps:
            x = x_new
            fval = fval_new
            grad = grad_func(x)
            Hess = hess_func(x)
            
            # Actualizamos el filtro retrospectivo
            s_k = x - Sk.dot(x)
            y_k = grad - Sk.dot(grad)
            if np.abs(np.dot(s_k, y_k)) > 1e-8:
                rho_rf = 1.0 / np.dot(s_k, y_k)
                Sk = (I - rho_rf * np.outer(s_k, y_k)).dot(Sk).dot(I - rho_rf * np.outer(y_k, s_k)) + rho_rf * np.outer(s_k, s_k)
            else:
                Sk = Sk
            
        if np.linalg.norm(grad) < eps:
            break
        
    return x, k

### 4. Método de Newton modificado

In [103]:
def newton_method(f,grad_f,H_f,x0,ite=1000, tol=1e-5):
    x = copy.deepcopy(x0)
    for i in range(ite):
        g = grad_f(x)
        H = H_f(x)
        # regularización para mejorar la estabilidad si es necesario
        eps = 1e-6
        H += np.eye(H.shape[0]) * eps
        # actualizar los pesos y sesgo usando el método de Newton modificado
        d = np.linalg.solve(H, -g)
        x += d
        
        #Calculamos el valor de la función en el punto en el que vamos
        norm = np.linalg.norm(grad_f(x))
        #print("Epoch %d - f(x) = %.4f - ||grad_f|| = %.4f" % (i, f(x),norm))
        
        # verificar si se ha alcanzado la tolerancia
        if  norm < tol:
            break
        
    return x,i

# Funciones

In [3]:
#Función Rosembrock 

def f_Rosenbrock(x):
    n = len(x)
    sum = 0
    for i in range(n-1):
        sum += 100*(x[i+1] - x[i]**2)**2 + (1 - x[i])**2
    return sum


def grad_rosenbrock(x):
    n = len(x)
    grad_f = np.zeros(n)
    grad_f[0] = 400*(x[0]**3) - 400*x[0]*x[1]+2*x[0]-2
    grad_f[n-1] = 200*x[n-1]-200*(x[n-2]**2) 
    
    for i in range(1,n-1):
        grad_f[i] = 400*(x[i]**3) + 202*x[i] - 400*x[i]*x[i+1]-200*(x[i-1]**2) - 2
    
    return grad_f

def H_rosenbrock(x):
    n = len(x)
    h_f = np.zeros((n,n))
    h_f[0,0] = 1200*(x[0]**2) - 400*x[1]+2
    h_f[0,1] = -400*x[0]
    h_f[n-1,n-2] = -400*x[n-2]
    h_f[n-1,n-1] = 200
    
    for i in range(1,n):
        h_f[i,i-1] = -400*x[i-1] 
    
    for i in range(1,n-1):
        h_f[i,i] = 1200*(x[i]**2) + 202 - 400*x[i+1]
        
    for i in range(0,n-1):
        h_f[i,i+1] = -400*x[i] 
        
    return h_f

In [130]:
#Función Wood

def f_wood(x):
    evaluation = 100*(x[0]**2 - x[1])**2 + (x[0] - 1)**2 + (x[2] - 1)**2 + 90*(x[2]**2 - x[3])**2 + 10.1*((x[1] - 1)**2 + (x[3] - 1)**2) + 19.8*(x[1] - 1)*(x[3] - 1) 
    return evaluation 


def grad_wood(x):
    Df = np.zeros(4)
    Df[0] = 400*x[0]**3 - 400*x[0]*x[1] + 2*(x[0]-1)
    Df[1] = 200*(x[1] - x[0]**2) + 20.2*(x[1] - 1) + 19.8*(x[3] - 1)
    Df[2] = 360*x[2]**2-360*x[2]*x[3]+2*x[2]-2
    Df[3] =  -180*(x[2]**2-x[3]) + 20.2*(x[3] - 1) + 19.8*(x[1] - 1)
    
    return Df

def H_wood(x):
    H_w = np.zeros((4,4))
    H_w[0,0] = 1200*x[0]**2 -400*x[1] + 2
    H_w[0,1] = -400*x[0]
    H_w[0,2] = 0
    H_w[0,3] = 0
    H_w[1,0] = -400*x[0]
    H_w[1,1] = 220.2
    H_w[1,2] = 0
    H_w[1,3] = 19.8
    H_w[2,0] = 0
    H_w[2,1] = 0
    H_w[2,2] = 720*x[2] - 360*x[3] + 2
    H_w[2,3] = -360*x[2]
    H_w[3,0] = 0 
    H_w[3,1] = 19.8
    H_w[3,2] = -360*x[2] 
    H_w[3,3] = 202.2
    
    return H_w

In [156]:
#Función de branin

b = 5.1/(4*math.pi*math.pi)
c = 5/math.pi
r = 6
s = 10
t = 1/(8*math.pi)

def f_branin(x):
    evaluation = (x[1]-b*x[0]**2 + c*x[0] - r)**2 + s*(1-t)*math.cos(x[0]) + s
    return evaluation

def grad_branin(x):
    g = np.zeros(len(x))
    g[0] = 2*(x[1] - b*x[0]**2 + c*x[0] - r)*(-2*b*x[0] + c) - s*(1-t)*math.sin(x[0])
    g[1] = 2*(x[1] - b*x[0]**2 + c*x[0] - r)
    return g

def H_branin(x):
    H = np.zeros((2,2))
    H[0,0] = 2*(-2*b*x[0] + c)*(-2*b*x[0] + c) + 2*(x[1] - b*x[0]**2 + c*x[0] - r)*(-2*b) - s*(1-t)*math.cos(x[0])
    H[1,1] = 2
    H[0,1] = H[1,0] = -4*b*x[0] + 2*c
    
    return H

# Experimentos

In [339]:
# Método de Dogleg, Función Rosenbrock
n = 100
m = 30
tim = 0
it = 0

for i in range(m):
    x0 = np.random.uniform(-2,2,n) + 1
    inicio = time.time()
    xmin,k = Trust_region_Dogleg(f_Rosenbrock,grad_rosenbrock,H_rosenbrock,x0,1,10,0.25,1e-8,5000)
    fin = time.time()
    tim = tim + (fin-inicio)
    it = it + k

tim = tim/m
it = it/m
print("Tiempo promedio: ", tim)
print("Iteraciones promedio: ", it)

Tiempo promedio:  2.466636864344279
Iteraciones promedio:  4726.3


In [338]:
# Método de Dogleg, Función wood
n = 4
m = 30
tim = 0
it = 0

for i in range(m):
    x0 = np.random.uniform(-2,2,n) + 1
    inicio = time.time()
    xmin,k = Trust_region_Dogleg(f_wood,grad_wood,H_wood,x0,1,10,0.25,1e-8,100)
    fin = time.time()
    tim = tim + (fin-inicio)
    it = it + k

tim = tim/m
it = it/m
print("Tiempo promedio: ", tim)
print("Iteraciones promedio: ", it)

Tiempo promedio:  0.00372008482615153
Iteraciones promedio:  86.73333333333333


In [337]:
# Método de Dogleg , Función branin
n = 2
m = 30
tim = 0
it = 0
x = np.array([math.pi,2.275])

for i in range(m):
    x0 = np.random.uniform(-2,2,n) + x
    inicio = time.time()
    xmin,k = Trust_region_Dogleg(f_branin,grad_branin,H_branin,x0,1,10,0.25,1e-8,100)
    fin = time.time()
    tim = tim + (fin-inicio)
    it = it + k

tim = tim/m
it = it/m
print("Tiempo promedio: ", tim)
print("Iteraciones promedio: ", it)

Tiempo promedio:  0.0014481147130330403
Iteraciones promedio:  34.53333333333333


In [345]:
# Método de Newton-Cauchy alterno , Función Rosenbrock
n = 100
m = 30
tim = 0
it = 0

for i in range(m):
    x0 = np.random.uniform(-2,2,n) + 1
    inicio = time.time()
    xmin,k = Trust_region_Newton_Cauchy(f_Rosenbrock,grad_rosenbrock,H_rosenbrock,x0,1,10,0.25,1e-5,1000)
    fin = time.time()
    tim = tim + (fin-inicio)
    it = it + k

tim = tim/m
it = it/m
print("Tiempo promedio: ", tim)
print("Iteraciones promedio: ", it)

Tiempo promedio:  0.45724849700927733
Iteraciones promedio:  999.0


In [344]:
# Método de Newton-Cauchy alterno , Función wood
n = 4
m = 30
tim = 0
it = 0

for i in range(m):
    x0 = np.random.uniform(-2,2,n) + 1
    inicio = time.time()
    xmin,k = Trust_region_Newton_Cauchy(f_wood,grad_wood,H_wood,x0,1,10,0.25,1e-6,500)
    fin = time.time()
    tim = tim + (fin-inicio)
    it = it + k

tim = tim/m
it = it/m
print("Tiempo promedio: ", tim)
print("Iteraciones promedio: ", it)

Tiempo promedio:  0.024936358133951824
Iteraciones promedio:  499.0


In [341]:
# Método de Newton-Cauchy alterno , Función branin
n = 2
m = 30
tim = 0
it = 0
x = np.array([math.pi,2.275])

for i in range(m):
    x0 = np.random.uniform(-2,2,n) + x
    inicio = time.time()
    xmin,k = Trust_region_Newton_Cauchy(f_branin,grad_branin,H_branin,x0,1,10,0.25,1e-6,100)
    fin = time.time()
    tim = tim + (fin-inicio)
    it = it + k

tim = tim/m
it = it/m
print("Tiempo promedio: ", tim)
print("Iteraciones promedio: ", it)

Tiempo promedio:  0.0037698745727539062
Iteraciones promedio:  66.16666666666667


In [179]:
# Método de Newton-modificado, Función Rosenbrock
n = 100
m = 30
tim = 0
it = 0

for i in range(m):
    x0 = np.random.uniform(-2,2,n) + 1
    inicio = time.time()
    xmin,k = newton_method(f_Rosenbrock,grad_rosenbrock,H_rosenbrock,x0,1000,1e-8)
    fin = time.time()
    tim = tim + (fin-inicio)
    it = it + k

tim = tim/m
it = it/m
print("Tiempo promedio: ", tim)
print("Iteraciones promedio: ", it)

Tiempo promedio:  0.2851104736328125
Iteraciones promedio:  748.3


In [180]:
# Método de Newton-modificado, Función wood
n = 4
m = 30
tim = 0
it = 0

for i in range(m):
    x0 = np.random.uniform(-2,2,n) + 1
    inicio = time.time()
    xmin,k = newton_method(f_wood,grad_wood,H_wood,x0,100,1e-8)
    fin = time.time()
    tim = tim + (fin-inicio)
    it = it + k

tim = tim/m
it = it/m
print("Tiempo promedio: ", tim)
print("Iteraciones promedio: ", it)

Tiempo promedio:  0.00041207472483317055
Iteraciones promedio:  9.5


In [181]:
# Método de Newton-modificado, Función branin
n = 2
m = 30
tim = 0
it = 0
x = np.array([math.pi,2.275])

for i in range(m):
    x0 = np.random.uniform(-2,2,n) + x
    inicio = time.time()
    xmin,k = newton_method(f_branin,grad_branin,H_branin,x0,100,1e-8)
    fin = time.time()
    tim = tim + (fin-inicio)
    it = it + k

tim = tim/m
it = it/m
print("Tiempo promedio: ", tim)
print("Iteraciones promedio: ", it)

Tiempo promedio:  0.0003091573715209961
Iteraciones promedio:  6.666666666666667


In [328]:
# Método RFTR, Función Rosenbrock
n = 100
m = 30
tim = 0
it = 0

for i in range(m):
    x0 = np.random.uniform(-2,2,n) + 1
    inicio = time.time()
    xmin,k = RFTR(f_Rosenbrock,grad_rosenbrock,H_rosenbrock, x0, maxiter=1000, eta1=0.05, eta2=0.9, gamma1=0.5, gamma2=2.0, eps=1e-5)
    fin = time.time()
    tim = tim + (fin-inicio)
    it = it + k

tim = tim/m
it = it/m
print("Tiempo promedio: ", tim)
print("Iteraciones promedio: ", it)

Tiempo promedio:  0.15028143723805745
Iteraciones promedio:  999.0


In [332]:
# Método RFTR, Función wood
n = 4
m = 30
tim = 0
it = 0

for i in range(m):
    x0 = np.random.uniform(-2,2,n) + 1
    inicio = time.time()
    xmin,k =  RFTR(f_wood,grad_wood,H_wood, x0, maxiter=1000, eta1=0.05, eta2=0.9, gamma1=0.5, gamma2=2.0, eps=1e-5)
    fin = time.time()
    tim = tim + (fin-inicio)
    it = it + k

tim = tim/m
it = it/m
print("Tiempo promedio: ", tim)
print("Iteraciones promedio: ", it)

Tiempo promedio:  0.020135887463887534
Iteraciones promedio:  882.1333333333333


In [334]:
# Método RFTR, Función branin
n = 2
m = 30
tim = 0
it = 0
x = np.array([math.pi,2.275])

for i in range(m):
    x0 = np.random.uniform(-2,2,n) + x
    inicio = time.time()
    xmin,k = RFTR(f_branin,grad_branin,H_branin, x0, maxiter=100, eta1=0.05, eta2=0.9, gamma1=0.5, gamma2=2.0, eps=1e-7)
    fin = time.time()
    tim = tim + (fin-inicio)
    it = it + k

tim = tim/m
it = it/m
print("Tiempo promedio: ", tim)
print("Iteraciones promedio: ", it)

Tiempo promedio:  0.0010746558507283529
Iteraciones promedio:  40.4
